In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [11]:
df = pd.read_csv('lobby-9-11-21-0900hrs-clean.csv')

In [12]:
df['time'] = pd.to_datetime(df['time'])

In [14]:
df

,temperature,time
0,29.9,2021-11-10 00:00:44.574000+00:00
1,29.9,2021-11-10 00:01:44.550000+00:00
2,29.9,2021-11-10 00:02:44.518000+00:00
3,29.9,2021-11-10 00:03:44.510000+00:00
4,29.9,2021-11-10 00:04:44.500000+00:00
...,...,...
96011,30.4,2022-01-16 23:55:04.057000+00:00
96012,30.4,2022-01-16 23:56:04.074000+00:00
96013,30.4,2022-01-16 23:57:04.077000+00:00
96014,30.4,2022-01-16 23:58:03.997000+00:00


In [19]:
train_size = int(len(df) * 0.66)
test_size = len(df) - train_size
train, test = df[0:train_size:], df[train_size:len(df):]

In [20]:
from keras.preprocessing.sequence import TimeseriesGenerator # Generates batches for sequence data
seq_size = length =  10 
batch_size = 1
train_generator = TimeseriesGenerator(train,train,length=length,batch_size=batch_size)
print("Total number of samples in the original training data = ", len(train))
print("Total number of samples in the generated data = ", len(train_generator)) 

Total number of samples in the original training data =  63370
Total number of samples in the generated data =  63360


In [25]:
validation_generator = TimeseriesGenerator(test, test, length=length ,batch_size=batch_size)
print("Total number of samples in the original training data = ", len(test))
print("Total number of samples in the generated data = ", len(validation_generator)) 

Total number of samples in the original training data =  32646
Total number of samples in the generated data =  32636


In [26]:
#Input dimensions are... (N x seq_size)
num_features = 1 #Univariate example

In [31]:
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(length, num_features)))
model.add(LSTM(50, activation='relu'))
#model.add(Dense(32))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

model.summary()
print('Train...')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 10, 50)            10400     
                                                                 
 lstm_3 (LSTM)               (None, 50)                20200     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________
Train...


In [32]:
model.fit_generator(generator=train_generator, verbose=2, epochs=100, validation_data=validation_generator)

C:\Users\ALMA12~1\AppData\Local\Temp/ipykernel_11480/416170021.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator, verbose=2, epochs=100, validation_data=validation_generator)


KeyError: 10